# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest F1 score, indicating the balance between precision and recall. Finally, the selected model is utilized to predict the outcomes for the current week's round of NRL matches. This process is iterative and cyclical, with the potential for revisiting earlier stages based on the model's performance. Let us proceed with this in-depth exploration.

## Set up Environment
This code segment is setting up the environment for the model training pipeline. It begins by importing sys and pathlib - Python libraries used for managing system parameters and file paths, respectively.

The code then updates the system path to include the "functions" directory. This allows for the import of custom modules `modelling_functions`, `model_properties`, and `training_config` which are stored in this directory. These modules contain custom functions and configuration settings that are critical for the later stages of data preprocessing, model training, and prediction.

Following this, the `project_root` variable is defined. This is achieved by using the pathlib library to establish the root directory of the project.

Finally, the `db_path` variable is constructed. This is the relative path to the SQLite database "footy-tipper-db.sqlite", which is located in the "data" directory of the project root. This path will be used for database connectivity throughout the pipeline.

In [1]:
import sys
import pathlib

sys.path.append("functions") 
import modelling_functions as mf
import model_properties as mp
import training_config as tc

# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [2]:
training_data = mf.get_training_data(db_path, 'sql/training_data.sql')
training_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,matchup_form,state_of_origin,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.020111e+10,1.0,Round 1,1.0,Final,1.584044e+09,1.584004e+09,CommBank Stadium,Sydney,21363.0,...,1.0,0.0,1510.543428,1496.559524,13.983904,0.513521,0.466892,0.019587,0.046629,8.542391
1,2.020111e+10,1.0,Round 1,2.0,Final,1.584122e+09,1.584083e+09,GIO Stadium,Canberra,10610.0,...,-1.0,0.0,1515.710577,1466.529411,49.181166,0.551101,0.409164,0.039735,0.141938,8.769688
2,2.020111e+10,1.0,Round 1,3.0,Final,1.584126e+09,1.584090e+09,Queensland Country Bank Stadium,Townsville,22459.0,...,1.0,0.0,1490.406034,1483.396877,7.009157,0.503694,0.476719,0.019587,0.026975,3.989305
3,2.020111e+10,1.0,Round 1,4.0,Final,1.584198e+09,1.584158e+09,McDonald Jones Stadium,Newcastle,10239.0,...,1.0,0.0,1484.839412,1486.546449,-1.707037,0.491400,0.489014,0.019587,0.002386,8.982643
4,2.020111e+10,1.0,Round 1,5.0,Final,1.584207e+09,1.584167e+09,Accor Stadium,Sydney,NaN,...,4.0,0.0,1505.190325,1506.316264,-1.125939,0.492220,0.488194,0.019587,0.004026,4.040292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,2.023111e+10,20.0,Round 20,3.0,Final,1.689442e+09,1.689406e+09,4 Pines Park,Sydney,13240.0,...,-3.0,0.0,1490.082965,1541.085361,-51.002395,0.417866,0.552283,0.029851,-0.134417,-2.128269
715,2.023111e+10,20.0,Round 20,4.0,Final,1.689450e+09,1.689414e+09,Sydney Cricket Ground,Sydney,12021.0,...,-3.0,0.0,1480.833645,1526.299892,-45.466247,0.425463,0.544686,0.029851,-0.119223,-3.406417
716,2.023111e+10,20.0,Round 20,5.0,Final,1.689523e+09,1.689480e+09,Go Media Stadium,Auckland,24012.0,...,-1.0,0.0,1528.936608,1528.030248,0.906359,0.495087,0.485326,0.019587,0.009761,3.185816
717,2.023111e+10,20.0,Round 20,6.0,Final,1.689524e+09,1.689488e+09,Kayo Stadium,Redcliffe,10065.0,...,0.0,0.0,1474.548764,1548.389112,-73.840348,0.391889,0.590720,0.017391,-0.198832,-11.470410


## Modelling
During the modelling phase, the `train_and_select_best_model` function, part of our `modelling_functions` module, is invoked. This function initiates the training of three distinct models: XGBoost, Random Forest, and Gradient Boosting Classifier. It takes as input the footy tipping data, predictor variables, the outcome variable, and several configuration settings like whether to use Recursive Feature Elimination (RFE), the number of cross-validation folds, and the optimization metric, all sourced from the `training_config` module.

The function first identifies categorical columns in the feature set for one-hot encoding, creating dummy variables for categorical features. Depending on the choice of using RFE, a feature elimination step may be included in the pipeline. Each model subsequently undergoes hyperparameter tuning using a genetic algorithm, facilitated by the `GASearchCV` function.

All the models are then trained and evaluated through cross-validation. The best model, or `footy_tipper`, is selected based on the superior performance on the chosen optimization metric. Additionally, a `LabelEncoder`(`label_encoder`), used to encode the categorical target variable, is returned. This LabelEncoder is specific to the model that performed best. The selected model, encapsulated in a pipeline with pre-processing steps and hyperparameter tuning, is now ready for the prediction phase.

In [3]:
footy_tipper, label_encoder = mf.train_and_select_best_model(
    training_data, tc.predictors, tc.outcome_var,
    tc.use_rfe, tc.num_folds, tc.opt_metric
)

footy_tipper


Model training: XGBClassifier
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	100   	0.699902	0.011206   	0.721785   	0.673067   
1  	134   	0.708845	0.00766237 	0.721785   	0.691171   
2  	149   	0.714925	0.00496811 	0.724563   	0.702292   
3  	142   	0.717924	0.00414014 	0.730128   	0.709256   
4  	137   	0.721756	0.0043809  	0.730128   	0.710655   
5  	144   	0.724292	0.00367263 	0.730128   	0.716191   
6  	148   	0.726284	0.00296829 	0.730128   	0.720387   
7  	132   	0.728013	0.0020222  	0.730128   	0.721785   
8  	138   	0.728836	0.00192816 	0.732887   	0.718998   
9  	131   	0.729669	0.00186613 	0.734314   	0.720396   
10 	128   	0.730281	0.000782162	0.734314   	0.72736    
11 	143   	0.730531	0.00132412 	0.734314   	0.725952   
12 	146   	0.730711	0.00132181 	0.734314   	0.725962   
13 	149   	0.731351	0.00177629 	0.734314   	0.724573   
14 	139   	0.732132	0.00198653 	0.734314   	0.723145   
15 	130   	0.733599	0.00127571 	0.734314   	0.730128   
16 	137   	0.7342

Pipeline(steps=[('one_hot_encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['round_name', 'venue_name',
                                                   'city', 'team_home',
                                                   'team_away',
                                                   'broadcast_channel1',
                                                   'broadcast_channel2',
                                                   'broadcast_channel3',
                                                   'game_day'])])),
                ('feature_elimination',
                 RFECV(cv=5,
                       estimator=XGBClassifier(base_score=Non...
                                        'learning_rate': <sklearn_genetic.space.space.Continuous object at 0x137427390>,
                                        'max_depth': <sklearn_genetic.space.space.Integer object at 0x1370d7c90>,
                                        'n_estimators': <sklearn_genetic.space.space.Integer object at 0x12156e850>,
                                        'subsample': <sklearn_genetic.space.space.Continuous object at 0x1370d7bd0>},
                            population_size=100, return_train_score=True,
                            scoring='accuracy'))])

### Display feature importance
The `get_feature_importance` function retrieves feature importances from a trained scikit-learn pipeline. It accounts for different transformations, such as one-hot encoding and recursive feature elimination. The function then returns a sorted DataFrame listing each feature alongside its respective importance, aiding in understanding the model's decision-making process.

In [4]:
# feature_importance_df = mp.get_feature_importances_from_pipeline(footy_tipper, tc.predictors)
# feature_importance_df

## Save Model
The `save_models` function stores the trained LabelEncoder and Pipeline objects to the disk. This allows for easy retrieval and reuse in future model prediction tasks, without the need to retrain these components. The objects are stored in a designated 'models' directory under the project root path, ensuring organized and consistent storage.

In [6]:
mf.save_models(label_encoder, footy_tipper, project_root)